In [1]:
from qiskit import *
from qiskit.circuit import ParameterVector
from qiskit.aqua.components.optimizers import ADAM,COBYLA,SPSA
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit.quantum_info import random_statevector
from qiskit.circuit.library import RXGate,RYGate,RZGate

In [3]:
#space to set parameters for the ansatz
num_qubits=4 #no of qubits for which the ansatz maybe applied. 4 in question given
gates=['rz','rx']#first item of the list selects parametrized gate for the odd block and 
                #second item on the list for the second block

In [4]:
def obj_fun(params,num_qubits=4,reps=4,gates=['rz','rx']):
    """function should take in a value of parameters, bind the given circuit to the said paramters and run the circuit
    then it should subtract the statvector of given circuit from the """
    num_qubits=num_qubits
    reps=reps
    circ=QuantumCircuit(num_qubits,num_qubits)
    x=ParameterVector('theta',length=num_qubits*2*reps)
    for i in range(len(gates)):
        if gates[i]=='rx':
            gates[i]=RXGate
        elif gates[i]=='ry':
            gates[i]=RYGate
        elif gates[i]=='rz':
            gates[i]=RZGate   
    t=0
    for _ in range(reps):
        for i in range(num_qubits):
            circ.append(gates[0](x[t]),[i])
            t+=1
        for i in range(num_qubits):
            for j in range(i+1,num_qubits):
                circ.cz(i,j)
        for i in range(num_qubits):
            circ.append(gates[1](x[t]),[i])
            t+=1
        circ.barrier()
    cirq=circ.bind_parameters({x:params})
    simulator = Aer.get_backend('statevector_simulator')
    # Execute and get counts
    result = execute(cirq, simulator,shots=1).result()
    svector = result.get_statevector(cirq)
    return np.sqrt(np.sum(np.power(np.abs(np.subtract(svector,rvector)),2)))

In [5]:
def obj_fun_optimizer(num_qubits, reps):
    def obj_fun_wrapper(params):
        return obj_fun(params,num_qubits,reps)##Takes reps from the arguement for parent function
    opt=COBYLA(tol=0.0001)
    return opt.optimize(num_qubits*2*reps,
               objective_function=obj_fun_wrapper,
                initial_point=np.random.rand(num_qubits*2*reps))

In [6]:
def main(num_qubits=4,gates=['rz','rx'],debug=False):
    rvector=random_statevector(2**num_qubits).data
    x,y=[],[]
    for i in range(1,25):
        res=obj_fun_optimizer(num_qubits,i)
        y.append(res[1])
        x.append(i)
        if debug==True:
            print(res[1],i)
        plt.plot(x,y)

In [ ]:
main(num_qubits,gates)# call the function with debug=True to see optimised cost for each depth